In [8]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import selenium
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchWindowException

In [9]:
# Selenium 옵션 설정
options = Options()
options.add_argument("--start-maximized")
options.add_experimental_option('detach', True)  # 스크립트가 종료된 후에도 브라우저가 닫히지 않도록 설정

# ChromeDriver 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# url Point
url = 'https://www.yes24.com/Product/Category/MonthWeekBestSeller?categoryNumber={}&pageNumber=1&pageSize=10&type=month&saleYear={}&saleMonth={}&sex={}&age={}'

categories = {
    "001001001": "가정 살림",
    "001001002": "자연과학",
    "001001003": "IT 모바일",
    "001001010": "역사",
    "001001011": "건강 취미",
    "001001016": "어린이",
    "001001019": "인문",
    "001001025": "경제 경영",
    "001001026": "자기계발",
    "001001046": "소설/시/희곡"
}

genders = ['F','M']
all_data = []

# Yes24 페이지 접속
for category in categories.keys() :
    for year in range(2023, 2024):
        for month in range(6, 13):
            if (year == 2023 and month < 6) or (year == 2024 and month > 6):
                continue
            current_month = month if month <= 12 else month - 12
            for gender in genders:
                for age in range(10, 61, 10): # 연령대 순회 설정
                    driver.get(url.format(category,year, month, gender, age))
                        # 책 정보 전부 로드될 때까지 대기
                    try:
                        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.gd_name')))  # 책제목

                        # 책 정보 추출
                        book_titles = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'a.gd_name')]  # 책제목
                        book_authors = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'span.authPub.info_auth')]  # 지은이
                        book_publisher = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'span.authPub.info_pub')]  # 출판사
                        book_prices = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'strong.txt_num')]  # 책가격
                        book_date = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'span.authPub.info_date')]  # 출판일
                        book_sales = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'span.saleNum')]  # 판매지수
                        book_ratings = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'span.rating_grade em.yes_b')]  # 평점
                        # book_volumes = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'span.gd_feature')]  # 권수

                        time.sleep(1)

                        # 각 리스트 길이가 동일한지 확인
                        # if not (len(book_titles) == len(book_authors) == len(book_publisher) == len(book_prices) == len(book_date) == len(book_sales) == len(book_ratings)):
                        #     print("The data length does not match, so it is filled with NaN values.")
                        #     continue

                        # 최대 길이를 기준으로 부족한 리스트에 NaN 값 채우기
                        max_length = max(len(book_titles), len(book_authors), len(book_publisher), len(book_prices), len(book_date), len(book_sales), len(book_ratings))

                        book_titles += [np.nan] * (max_length - len(book_titles))
                        book_authors += [np.nan] * (max_length - len(book_authors))
                        book_publisher += [np.nan] * (max_length - len(book_publisher))
                        book_prices += [np.nan] * (max_length - len(book_prices))
                        book_date += [np.nan] * (max_length - len(book_date))
                        book_sales += [np.nan] * (max_length - len(book_sales))
                        book_ratings += [np.nan] * (max_length - len(book_ratings))

                        data = {
                            'Title': book_titles,
                            'Author': book_authors,
                            'Publisher': book_publisher,
                            'Price': book_prices,
                            'Date' : book_date,
                            'Sales Index': book_sales,
                            'Rating': book_ratings,
                            # 'Volumes': book_volumes
                        }

                        # 데이터프레임 생성 및 칼럼 데이터 값 정리
                        df = pd.DataFrame(data)
                        
                        if year == 2023:
                            df['Year'] = 2023
                        else:
                            df['Year'] = 2024
                        df['Month'] = current_month
                        df['Category'] = categories[category]
                        df['Author'] = df['Author'].apply(lambda x:x.split(' 저')[0])
                        df['Gender'] = '1' if gender == 'F' else '0'
                        df['Sales Index'] = df['Sales Index'].apply(lambda x:x.split(' ')[1].replace(',',''))
                        df['Price'] = df['Price'].str.replace('[,원]', '', regex=True).astype(int)
                        df['Age'] = age
                        # df['Volumes'] = df['Volumes'].apply(lambda x: x.split('전')[1].split('권')[0] if '전' in x and '권' in x else np.nan).astype(float)

                        all_data.append(df)

                    except Exception as e:
                        print('예외 발생 : {e}')
driver.quit()

# DataFrame으로 합치기
all_data_df = pd.concat(all_data, ignore_index=True)

Error : All arrays must be of the same length
Error : All arrays must be of the same length
Error : All arrays must be of the same length
Error : All arrays must be of the same length
Error : All arrays must be of the same length
Error : All arrays must be of the same length
Error : All arrays must be of the same length
Error : All arrays must be of the same length
Error : All arrays must be of the same length
Error : All arrays must be of the same length
Error : All arrays must be of the same length
Error : All arrays must be of the same length
Error : All arrays must be of the same length
Error : All arrays must be of the same length
Error : All arrays must be of the same length
Error : All arrays must be of the same length


ValueError: No objects to concatenate

In [12]:
# Selenium 옵션 설정
options = Options()
options.add_argument("--start-maximized")
options.add_experimental_option('detach', True)  # 스크립트가 종료된 후에도 브라우저가 닫히지 않도록 설정

# ChromeDriver 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# url Point
url = 'https://www.yes24.com/Product/Category/MonthWeekBestSeller?categoryNumber={}&pageNumber=1&pageSize=10&type=month&saleYear={}&saleMonth={}&sex={}&age={}'

categories = {
    "001001001": "가정 살림",
    "001001002": "자연과학",
    "001001003": "IT 모바일",
    "001001010": "역사",
    "001001011": "건강 취미",
    "001001016": "어린이",
    "001001019": "인문",
    "001001025": "경제 경영",
    "001001026": "자기계발",
    "001001046": "소설/시/희곡"
}

genders = ['F', 'M']
all_data = []

# Yes24 페이지 접속
for category in categories.keys():
    for year in range(2023, 2024):
        for month in range(6, 13):
            current_month = month if month <= 12 else month - 12
            for gender in genders:
                for age in range(10, 61, 10):  # 연령대 순회 설정
                    driver.get(url.format(category, year, month, gender, age))
                    # 책 정보 전부 로드될 때까지 대기
                    try:
                        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.gd_name')))  # 책제목

                        # 책 정보 추출
                        book_titles = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'a.gd_name')]  # 책제목
                        book_authors = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'span.authPub.info_auth')]  # 지은이
                        book_publisher = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'span.authPub.info_pub')]  # 출판사
                        book_prices = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'strong.txt_num')]  # 책가격
                        book_date = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'span.authPub.info_date')]  # 출판일
                        book_sales = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'span.saleNum')]  # 판매지수
                        book_ratings = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'span.rating_grade em.yes_b')]  # 평점
                        book_volumes = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'span.gd_feature')]  # 권수

                        time.sleep(1)

                        # 최대 길이를 기준으로 부족한 리스트에 NaN 값 채우기
                        max_length = max(len(book_titles), len(book_authors), len(book_publisher), len(book_prices), len(book_date), len(book_sales), len(book_ratings))

                        book_titles += [np.nan] * (max_length - len(book_titles))
                        book_authors += [np.nan] * (max_length - len(book_authors))
                        book_publisher += [np.nan] * (max_length - len(book_publisher))
                        book_prices += [np.nan] * (max_length - len(book_prices))
                        book_date += [np.nan] * (max_length - len(book_date))
                        book_sales += [np.nan] * (max_length - len(book_sales))
                        book_ratings += [np.nan] * (max_length - len(book_ratings))
                        book_volumes += [np.nan] * (max_length - len(book_volumes))

                        data = {
                            'Title': book_titles,
                            'Author': book_authors,
                            'Publisher': book_publisher,
                            'Price': book_prices,
                            'Date': book_date,
                            'Sales Index': book_sales,
                            'Rating': book_ratings,
                            'Volumes': book_volumes
                        }

                        # 데이터프레임 생성 및 칼럼 데이터 값 정리
                        df = pd.DataFrame(data)
                        
                        df['Year'] = year
                        df['Month'] = current_month
                        df['Category'] = categories[category]
                        df['Author'] = df['Author'].apply(lambda x: x.split(' 저')[0])
                        df['Gender'] = '1' if gender == 'F' else '0'
                        df['Sales Index'] = df['Sales Index'].apply(lambda x: x.split(' ')[1].replace(',', '') if isinstance(x, str) else x)
                        df['Price'] = df['Price'].str.replace('[,원]', '', regex=True).astype(float)
                        df['Age'] = age
                        df['Volumes'] = df['Volumes'].apply(lambda x: x.split('전')[1].split('권')[0] if isinstance(x, str) and '전' in x and '권' in x else np.nan).astype(float)

                        all_data.append(df)

                    except Exception as e:
                        print(f"Error : {e}")

driver.quit()

# DataFrame으로 합치기
if all_data:
    all_data_df = pd.concat(all_data, ignore_index=True)
else:
    print("No data collected.")


In [11]:
# CSV 파일로 저장
all_data_df.to_csv('Yes24_books_data_sample.csv', index=False, encoding='utf-8-sig')

In [2]:
import pandas as pd
df = pd.read_csv('Yes24_books_data.csv', encoding='utf-8-sig')

In [3]:
# 전처리 Step
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14930 entries, 0 to 14929
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        14930 non-null  object 
 1   Author       14930 non-null  object 
 2   Publisher    14930 non-null  object 
 3   Price        14930 non-null  int64  
 4   Date         14930 non-null  object 
 5   Sales Index  14930 non-null  int64  
 6   Rating       14930 non-null  float64
 7   Year         14930 non-null  int64  
 8   Month        14930 non-null  int64  
 9   Category     14930 non-null  object 
 10  Gender       14930 non-null  int64  
 11  Age          14930 non-null  int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 1.4+ MB


In [ ]:
# 데이터 타입 변환
df['Publisher'] = df['Publisher'].astype(str).str

In [ ]:
# 데이터 범주화
price_bins = [0, 5000, 10000, 15000, 20000, df['Price'].max()]
data['Price_Binned'] = pd.cut(df['Price'], bins = price_bins, labels=['0-0.5', '0.5-1.0','1.0-1.5', '1.5-2.0', '2.0++'])

rating_bins = [5,7,8,9, df['Rating'].max]
data['Rating_Binned'] = pd.cut(df['Rating'], bins = rating_bins, labels = ['5','5-6','4-4.5','4.5-5'])

age_bins = [0, 11, 21, 31, 41, 51, 61, 70]
df['age_labels'] = pd.cut(df['Age'], bins = age_bins, labels = ['Teenager', 'Young Adult', 'Adult', 'Middle Age', 'Late Middle Age', 'Senior'])

In [37]:
df.head()

,Title,Author,Publisher,Price,Date,Sales Index,Rating,Year,Month,Category,Gender,Age
0,팬 하나로 충분한 두 사람 식탁,국가비,달,31500,2023년 06월,20880,9.4,2023,6,가정 살림,1,10
1,집에서 운영하는 작은 빵집 SOFT BREAD,호야,더테이블,29700,2023년 06월,21522,9.9,2023,6,가정 살림,1,10
2,베이킹은 과학이다,"나카야마 히로노리, 기무라 마키코",터닝포인트,20700,2017년 11월,11133,9.5,2023,6,가정 살림,1,10
3,맛있어서 지속 가능한 디디미니 다이어트 레시피,미니 박지우,빅피시,16920,2022년 05월,7632,9.9,2023,6,가정 살림,1,10
4,"사춘기 딸에게 힘이 되어주는, 부모의 말 공부",이현정,포레스트북스,15750,2023년 04월,14067,9.6,2023,6,가정 살림,1,10


In [36]:
missing_values_count = df.isnull().sum()
print(missing_values_count)

Title          0
Author         0
Publisher      0
Price          0
Date           0
Sales Index    0
Rating         0
Year           0
Month          0
Category       0
Gender         0
Age            0
dtype: int64


In [39]:
# 데이터 탐색적 분석 Step

# 통계
# df.describe()
# df['Category'].value_counts()
# df['Age'].value_counts()
# df['Publisher'].value_counts()
# df['Gender'].value_counts()
df['Price'].value_counts()

Price
16200     1254
19800     1214
15300      778
15750      777
16020      671
          ... 
39600        1
19620        1
21060        1
128700       1
160650       1
Name: count, Length: 157, dtype: int64

In [ ]:
# 데이터 범주화
price_bins = [0, 500000, 1000000, 1500000, 2000000, data['Price'].max()]

In [ ]:
# 상관관계 분석
import seaborn as sns
import matplotlib.pyplot as plt
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True)
plt.show()

In [ ]:
# 카테고리화 시각화때 사용
# if age == 10:
#     df['Age'] = 'Teenager'
# elif age == 20:
#     df['Age'] = 'Young Adult'
# elif age == 30:
#     df['Age'] = 'Adult'
# elif age == 40:
#     df['Age'] = 'Middle Age'
# elif age == 50:
#     df['Age'] = 'Late Middle Age'
# else:
#     df['Age'] = 'Senior'

In [17]:
# 향후 Update - User 편의기능 추가 - 검색기능. Type1
query = input('검색어 입력: ')
yes24_url = f'https://www.yes24.com/Product/Search?domain=ALL&query={query}'
millie_url = f'https://www.millie.co.kr/v3/search/result/{query}?type=total&contentcode=0&searchBack=y&nav_hidden=y&category=0'

driver = webdriver.Chrome()

driver.get(yes24_url)
time.sleep(2)

first_book_link = driver.find_element(By.CSS_SELECTOR, 'a.gd_name')
first_book_link.click()
time.sleep(2)

review_more = driver.find_elements(By.CSS_SELECTOR, 'span.review_more')
for more in review_more[:2]:
        more.click()
        time.sleep(1)

coments = driver.find_elements(By.CSS_SELECTOR, 'div.review_cont')
for coment in enumerate(coments):
    driver.find_elements(By.CLASS_NAME, 'spanp.txt').text()
    for txt in coment[:2]:
        print(txt.text)

driver.quit()

AttributeError: 'list' object has no attribute 'text'

In [24]:
# 향후 Update - User 편의기능 추가 - 검색기능. Type2
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

query = input('검색어 입력: ')
yes24_url = f'https://www.yes24.com/Product/Search?domain=ALL&query={query}'
millie_url = f'https://www.millie.co.kr/v3/search/result/{query}?type=total&contentcode=0&searchBack=y&nav_hidden=y&category=0'

driver = webdriver.Chrome()

try:
    driver.get(yes24_url)
    time.sleep(2)

    first_book_link = driver.find_element(By.CSS_SELECTOR, 'a.gd_name')
    first_book_link.click()
    time.sleep(2)

    review_more = driver.find_elements(By.CSS_SELECTOR, 'span.review_more')
    for more in review_more[:2]:
        more.click()
        time.sleep(1)

    coments = driver.find_elements(By.CSS_SELECTOR, 'div.review_cont')
    for coment in coments[:2]:  # 처음 두 개의 댓글만 순회
        print(coment.text)

finally:
    driver.quit()


KeyboardInterrupt: 

In [23]:
# 향후 Update - User 편의기능 추가 - 검색기능. Type3
keyword = input("책 제목을 입력하세요: ")
search_url = f'https://search.kyobobook.co.kr/search?keyword={keyword}&gbCode=TOT&target=total'
driver = webdriver.Chrome()
driver.get(search_url)
element = driver.find_element(By.CLASS_NAME, 'auto_overflow_inner').click()

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
items = soup.select('.comment_text')
# 추천수 상위 3개의 리뷰만 가져오기
reviews = items[:2]

# 결과 출력
if len(reviews) == 0:
    print("리뷰가 없습니다")
else:
    for review in reviews:
        print(review.get_text(strip=True))

그의 운명은 지극히 비참할 것입니다. 독재자.
읽기전부터 재밌다ㅋㅋㅋ2년전 체한 기운이 싹내려감
